# Load Data

In [60]:
import music21
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import mnet

Create path to access local files

In [61]:
'''
Creat local corpus with access pieces
'''

localCorpus = music21.corpus.corpora.LocalCorpus()
localCorpus.addPath('../library')
music21.corpus.cacheMetadata()

/opt/anaconda3/envs/music-network-env/lib/python3.7/site-packages/music21/corpus/corpora.py: WARNING: local metadata cache: starting processing of paths: 12
/opt/anaconda3/envs/music-network-env/lib/python3.7/site-packages/music21/corpus/corpora.py: WARNING: cache: filename: /var/folders/j6/2q7hftds7vb8_lsfy50l8lzw0000gn/T/music21/local.p.gz
bundles.py: WARNING: MetadataBundle Modification Time: 1587701543.688641
bundles.py: WARNING: Skipped 12 sources already in cache.
/opt/anaconda3/envs/music-network-env/lib/python3.7/site-packages/music21/corpus/corpora.py: WARNING: cache: writing time: 0.025 md items: 12

/opt/anaconda3/envs/music-network-env/lib/python3.7/site-packages/music21/corpus/corpora.py: WARNING: cache: filename: /var/folders/j6/2q7hftds7vb8_lsfy50l8lzw0000gn/T/music21/local.p.gz
caching.py: WARNING: cache: final writing time: 0.025 seconds


Save Telemann Fantasie No. 1 as a stream object

In [62]:
s = music21.corpus.parse('telemannfantasie1.xml')

Inspect stream to see where desired parts are located

Save Solo Flute part

In [63]:
flute = s[5]

# Discreet Sections Pipeline

Define groups as list where each entry is the measure that a new section starts. Need the last entry to be "end"

In [64]:
flute_notes =flute.recurse().notes

In [65]:
grouping = [1, 5, 11, 27, 37, 49, 61, 75, "end"]

Use the convert_grouping function to convert note list and groups into a list of nodes. Will also return list of section-transition edges

In [66]:
nodelst_group, transition_edges=mnet.convert_grouping(flute_notes, grouping)


Create Graph

In [68]:
g_group=mnet.create_graph(nodelst_group)
#Write to .gexf
nx.write_gexf(g_group, "grouped_composition.gexf")

Generate Random Walk

In [9]:
randomwalk_group=mnet.generate_randomwalk(g_group)

Convert random walk to back to music.

Uses conversion function group_strto16thnote, which takes a group pitch string and converts it to a note with length of 16th note. Notes preceeding transition edges will be half notes

In [10]:
new_composition_group = mnet.convert_to_stream(randomwalk_group, mnet.group_strto16thnote)
#Write to MIDI
#new_composition_group.write('midi', "new_composition_group.mid")

In [11]:
g_group.nodes

NodeView(('1 A4', '1 B4', '1 E4', '1 C#5', '1 E5', '1 F#5', '1 D5', '1 G#5', '1 A5', '5 D5', '5 C#5', '5 F#5', '5 A5', '5 B4', '5 E5', '5 D4', '5 A4', '5 D#4', '5 E4', '5 G#4', '5 G#5', '5 B5', '11 A4', '11 D5', '11 C#5', '11 F#5', '11 B4', '11 E5', '11 E4', '11 G#4', '11 A5', '11 F#4', '11 G#5', '11 C#6', '11 B5', '11 A#4', '11 G5', '11 G4', '11 D4', '27 A5', '27 E5', '27 C#5', '27 A4', '27 G5', '27 F#5', '27 D5', '27 B5', '27 D#5', '27 B4', '27 F#4', '27 G#4', '27 E4', '27 D#4', '37 C#5', '37 D5', '37 E5', '37 B4', '37 A4', '37 G#4', '37 E4', '37 F#5', '37 G#5', '37 A5', '37 D#5', '37 F#4', '49 C#5', '49 D5', '49 E5', '49 B4', '49 A4', '49 G#4', '49 E4', '49 F#5', '49 G#5', '49 A5', '49 D#5', '49 F#4', '61 D5', '61 E5', '61 F#5', '61 C#5', '61 B4', '61 A4', '61 A5', '61 G#5', '61 E4', '61 G#4', '75 D5', '75 E5', '75 F#5', '75 C#5', '75 B4', '75 A4', '75 A5', '75 G#5', '75 E4', '75 G#4', 'start', 'end'))

## Alter Weight of Transition Nodes

Currently all transition edges have a weight of 1. This can be altered through degree_increase()

In [57]:
transition_edges

[('1 A4', '5 D5'),
 ('5 E4', '11 A4'),
 ('11 A4', '27 A5'),
 ('27 E4', '37 C#5'),
 ('37 E4', '49 C#5'),
 ('49 E4', '61 D5'),
 ('61 A4', '75 D5')]

In [58]:
graph_adjusted1 = mnet.degree_increase(g_group, [transition_edges[2]], increase=10)

The weight of the outgoing edge affects how much time the random walker will stay in a givin section. By changing this weight, you have some degree of control over how long a section will go for. 

Page Rank

In [43]:
g_weighted = mnet.convert_to_weighted(g_group)

In [14]:
ranking = nx.pagerank(g_weighted)

In [15]:
len(ranking)

99

Module Leaving Probability
    
    ni : number of nodes in module i

    alpha : node label

    tau: probability of teleportation  --> very small could make constant

    p_alpha: probability of being at node (pagerank)

    w_alpha_beta : weight of link from node alpha to node beta

In [16]:
def exit_probability(steps, graph, module, ranking, tau=.15):
    n = len(graph.nodes)
    ni = len(module.nodes)
    
    #teleportation term - may remove
    sum_rank=0
    for alpha in module.nodes:
        sum_rank += ranking[alpha]
        
    #exit term
    sum_exit_rank_weight = 0
    for edge in graph.edges():
        if edge[0] in module.nodes and edge[1] not in module.nodes:
            weight=graph.get_edge_data(edge[0], edge[1])['weight']
            print(weight)
            print(ranking[edge[0]])
            print(edge[0])
            print(edge[1])
            sum_exit_rank_weight += ranking[edge[0]] * weight
    
    step_exit_probability = tau * ((n-ni)/(n-1)) * sum_rank + (1-tau) * sum_exit_rank_weight
    print('step exit probability', step_exit_probability)
    exit_probability = step_exit_probability * ((1-step_exit_probability) ** (steps-1))
    
    return exit_probability
        
    

In [52]:
g_group = mnet.degree_increase(g_weighted, [('1 A4', '5 D5')], 30)

Isolate Module

In [44]:
module = g_weighted.subgraph(['1 A4', \
        '1 B4', '1 E4', '1 C#5', '1 E5', '1 A5', \
                              '1 D5', '1 F#5', '1 G#5'])

Calc pagerank -liklihood a random walker will be at a particular node

In [47]:
ranking = nx.pagerank(g_weighted)

In [48]:
ranking['5 D5']

0.013991789218514624

In [53]:
exit_1=exit_probability(1, g_weighted, module, ranking, tau=.5)
exit_1

0.14285714285714285
0.02269908534873689
1 A4
5 D5
step exit probability 0.03986049085093134


0.03986049085093134

In [56]:
exit_2=exit_probability(2, g_weighted, module, ranking, tau=.5)
exit_2

0.14285714285714285
0.02269908534873689
1 A4
5 D5
step exit probability 0.03986049085093134


0.038271632120054154

In [55]:
exit_3=exit_probability(20, g_weighted, module, ranking, tau=.01)
exit_3

0.14285714285714285
0.02269908534873689
1 A4
5 D5
step exit probability 0.003975081765841791


0.0036853601438684106

### Ground Truth for length probabilities

Isolate subgraph

In [23]:
subgraph = g_group.subgraph(['1 A4', \
        '1 B4', '1 E4', '1 C#5', '1 E5', '1 A5', \
                              '1 D5', '1 F#5', '1 G#5', '5 D5'])

In [29]:
arr = mnet.gen_path_histogram(subgraph,  '1 A4',  ('1 A4', '5 D5'), outgoing_edge_weight=10, additional_depth=3)


[('1 F#5', '1 D5'), ('1 F#5', '1 G#5'), ('1 D5', '1 E5'), ('1 D5', '1 B4'), ('1 G#5', '1 A5'), ('1 E5', '1 A4'), ('1 E5', '1 F#5'), ('1 E5', '1 C#5'), ('1 A4', '1 A4'), ('1 A4', '1 B4'), ('1 A4', '1 E4'), ('1 A4', '1 C#5'), ('1 A4', '1 E5'), ('1 A4', '1 A5'), ('1 A4', '5 D5'), ('1 C#5', '1 A4'), ('1 C#5', '1 D5'), ('1 C#5', '1 E5'), ('1 A5', '1 E5'), ('1 A5', '1 A4'), ('1 B4', '1 A4'), ('1 B4', '1 C#5'), ('1 E4', '1 A4')]
current path length is  0
n is  1 A4
weight is  0.038461538461538464
n is  1 B4
weight is  0.11538461538461539
n is  1 E4
weight is  0.19230769230769232
n is  1 C#5
weight is  0.07692307692307693
n is  1 E5
weight is  0.07692307692307693
n is  1 A5
weight is  0.07692307692307693
n is  5 D5
weight is  0.4230769230769231
current path length is  1
n is  1 A4
weight is  0.0014792899408284025
n is  1 B4
weight is  0.004437869822485207
n is  1 E4
weight is  0.007396449704142012
n is  1 C#5
weight is  0.002958579881656805
n is  1 E5
weight is  0.002958579881656805
n is  1 A5

n is  1 C#5
weight is  0.0002366863905325444
current path length is  4
n is  1 A4
weight is  1.5804379709705156e-06
n is  1 B4
weight is  4.741313912911547e-06
n is  1 E4
weight is  7.902189854852578e-06
n is  1 C#5
weight is  3.160875941941031e-06
n is  1 E5
weight is  3.160875941941031e-06
n is  1 A5
weight is  3.160875941941031e-06
n is  5 D5
weight is  1.738481768067567e-05
current path length is  4
n is  1 D5
weight is  8.21827744904668e-05
n is  1 G#5
weight is  4.10913872452334e-05
current path length is  4
n is  1 A4
weight is  9.131419387829645e-05
n is  1 D5
weight is  5.478851632697787e-05
n is  1 E5
weight is  1.826283877565929e-05
current path length is  4
n is  1 A4
weight is  5.470746822590247e-07
n is  1 B4
weight is  1.641224046777074e-06
n is  1 E4
weight is  2.735373411295123e-06
n is  1 C#5
weight is  1.0941493645180493e-06
n is  1 E5
weight is  1.0941493645180493e-06
n is  1 A5
weight is  1.0941493645180493e-06
n is  5 D5
weight is  6.017821504849271e-06
current pa

current path length is  4
n is  1 A4
weight is  4.551661356395085e-05
n is  1 C#5
weight is  6.827492034592626e-05
current path length is  4
n is  1 A4
weight is  0.00018965255651646185
current path length is  4
n is  1 A4
weight is  4.214501255921375e-05
n is  1 D5
weight is  2.528700753552825e-05
n is  1 E5
weight is  8.429002511842749e-06
current path length is  4
n is  1 A4
weight is  9.482627825823093e-06
n is  1 F#5
weight is  2.844788347746928e-05
n is  1 C#5
weight is  3.7930511303292373e-05
current path length is  4
n is  1 E5
weight is  5.05740150710565e-05
n is  1 A4
weight is  2.528700753552825e-05
current path length is  4
current path length is  4
n is  1 A4
weight is  4.551661356395084e-05
n is  1 B4
weight is  0.00013654984069185253
n is  1 E4
weight is  0.0002275830678197542
n is  1 C#5
weight is  9.103322712790169e-05
n is  1 E5
weight is  9.103322712790169e-05
n is  1 A5
weight is  9.103322712790169e-05
n is  5 D5
weight is  0.0005006827492034593
current path length 

In [27]:
arr

array([0.        , 0.0625    , 0.00390625, 0.0323839 , 0.01086867])